In [ ]:
from collections import OrderedDict
import torch

import sys
sys.path.append("..")
import gb
from gb.exutil import *

In [ ]:
from gb.model import GraphSequential, PreprocessA, PreprocessX, PreprocessAUsingXMetric, GCN, RGCN, ProGNN, GNNGuard, \
    GRAND, MLP, SoftMedianPropagation
from gb.pert import sp_edge_diff_matrix, sp_feat_diff_matrix
from gb.torchext import mul
from gb import metric, preprocess

In [ ]:
dataset = "cora"
A, X, y = gb.data.get_dataset(dataset)
N, D = X.shape
C = y.max().item() + 1
train_nodes, val_nodes, test_nodes = gb.data.get_splits(y)[0]  # [0] = select first split

A = A.cuda()
X = X.cuda()
y = y.cuda()

In [ ]:
ptb_rate = 0.10

In [ ]:
budget = int(ptb_rate * (A.cpu().numpy().sum() // 2))
budget

In [ ]:
ptb_value = str(int(ptb_rate*100))
ptb_value

In [ ]:
accuracy_dict = {}
accuracy_dict['GCN']={}
accuracy_dict['GRAND']={}
accuracy_dict['GNNGuard']={}
accuracy_dict['GCNSVD']={}
accuracy_dict['ProGNN']={}

In [ ]:
!nvidia-smi

## GCN

In [ ]:
torch.manual_seed(42)

fit_kwargs = dict(lr=1e-2, weight_decay=5e-4)

def make_model():
    return gb.model.GCN(n_feat=D, n_class=C, hidden_dims=[64], dropout=0.5).cuda()

aux_model = make_model()
aux_model.fit((A, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs)

In [ ]:
clean_accuracy = gb.metric.accuracy(aux_model(A, X)[test_nodes], y[test_nodes]).item()
accuracy_dict['GCN']['clean']=clean_accuracy

print("Clean test acc:   ", clean_accuracy)


In [ ]:
clean_vals = aux_model.feature_vals

In [ ]:
for k,v in clean_vals.items():
    print(v.shape)
    clean_vals[k]=v.cpu()

In [ ]:
import numpy as np
np.savez(f'feature_vals/gcn_clean_{ptb_value}.npz', **clean_vals)

In [ ]:
aux_model.feature_vals

### Poisoning global

In [ ]:
def loss_fn(A_flip):
    A_pert = A + A_flip * (1 - 2 * A)

    ########### Meta-Attack w/ Adam ##########
    model = make_model()
    model.fit((A_pert, X), y, train_nodes, val_nodes, progress=False,\
              **fit_kwargs, differentiable=A_pert.requires_grad)
    ##########################################

    scores = model(A_pert, X)
    return gb.metric.margin(scores[test_nodes, :], y[test_nodes]).tanh().mean()

def grad_fn(A_flip):
    return torch.autograd.grad(loss_fn(A_flip), A_flip)[0]

In [ ]:
########### PGD for Meta-Attack ##########
pert, _ = gb.attack.proj_grad_descent(A.shape, True, A.device, budget, grad_fn, loss_fn, \
                                      base_lr=0.01, grad_clip=1)

In [ ]:
A_pert = A + gb.pert.edge_diff_matrix(pert, A)

In [ ]:
print("Adversarial edges:", pert.shape[0])

In [ ]:
pois_model = make_model()
pois_model.fit((A_pert, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs)
pois_accuracy = gb.metric.accuracy(pois_model(A_pert, X)[test_nodes], y[test_nodes]).item()
accuracy_dict['GCN']['pois']=pois_accuracy

print("Poisoned test acc:", pois_accuracy)

In [ ]:
pois_vals=pois_model.feature_vals

In [ ]:
for k,v in pois_vals.items():
    print(v.shape)
    pois_vals[k]=v.cpu()

In [ ]:
import numpy as np
np.savez('feature_vals/gcn_gp_'+ptb_value+'.npz', **pois_vals)

### Evasion global

In [ ]:
def loss_fn(A_flip):
    A_pert = A + A_flip * (1 - 2 * A)

    ############### Aux-Attack ###############
    model = aux_model

    scores = model(A_pert, X)
    return gb.metric.margin(scores[test_nodes, :], y[test_nodes]).tanh().mean()

In [ ]:
########### PGD for Aux-Attack ###########
pert, _ = gb.attack.proj_grad_descent(A.shape, True, A.device, budget, grad_fn, loss_fn,\
                                      base_lr=0.1)

In [ ]:
A_pert = A + gb.pert.edge_diff_matrix(pert, A)
print("Adversarial edges:", pert.shape[0])
evas_accuracy = gb.metric.accuracy(aux_model(A_pert, X)[test_nodes], y[test_nodes]).item()
accuracy_dict['GCN']['evas'] = evas_accuracy

print("Evasion test acc: ", evas_accuracy)

In [ ]:
aux_model(A_pert,X)
evasion_vals=aux_model.feature_vals

In [ ]:
evasion_vals

In [ ]:
for k,v in evasion_vals.items():
    print(v.shape)
    evasion_vals[k]=v.cpu()

In [ ]:
import numpy as np
np.savez('feature_vals/gcn_ge_'+ptb_value+'.npz', **evasion_vals)

In [ ]:
del clean_vals
del evasion_vals
del pois_vals

## GCN-SVD

In [ ]:
rank = 50
fit_kwargs = dict(lr=1e-2, weight_decay=5e-4)

def make_model():
    return gb.model.GraphSequential(OrderedDict(
        low_rank=gb.model.PreprocessA(lambda A: gb.preprocess.low_rank(A, rank)),
        gcn=gb.model.GCN(n_feat=D, n_class=C, hidden_dims=[64], dropout=0.5)
    )).cuda()

aux_model = make_model()
aux_model.fit((A, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs)

A_low_rank = aux_model.low_rank(A)
A_weights = gb.metric.eigenspace_alignment(A, rank)

In [ ]:
clean_accuracy = gb.metric.accuracy(aux_model(A, X)[test_nodes], y[test_nodes]).item()
accuracy_dict['GCNSVD']['clean']=clean_accuracy

print("Clean test acc:   ", clean_accuracy)

In [ ]:
clean_vals = aux_model.gcn.feature_vals

In [ ]:
for k,v in clean_vals.items():
    print(v.shape)
    clean_vals[k]=v.cpu()

In [ ]:
import numpy as np
np.savez(f'feature_vals/gcnsvd_clean_{ptb_value}.npz', **clean_vals)

### Poisoning global

In [ ]:
def loss_fn(A_flip):
    A_diff = A_flip * (1 - 2 * A)

    ############# w/ weights #############
    #A_diff = A_diff * A_weights
    ######################################

    A_pert = A_low_rank + A_diff


    ############# Meta-Attack ############
    model = make_model().sub(exclude=["low_rank"])
    model.fit((A_pert, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs, differentiable=A_pert.requires_grad)
    ######################################

    scores = model(A_pert, X)
    return gb.metric.margin(scores[test_nodes, :], y[test_nodes]).tanh().mean()

In [ ]:
def grad_fn(A_flip):
    return torch.autograd.grad(loss_fn(A_flip), A_flip)[0]

In [ ]:
########## PGD for Meta-Attack ##########
pert, _ = gb.attack.proj_grad_descent(A.shape, True, A.device, budget, grad_fn, loss_fn,\
                                      base_lr=0.1, grad_clip=0.1)
A_pert = A + gb.pert.edge_diff_matrix(pert, A)

In [ ]:
pois_model = make_model()
pois_model.fit((A_pert, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs)
pois_accuracy = gb.metric.accuracy(pois_model(A_pert, X)[test_nodes], y[test_nodes]).item()
accuracy_dict['GCNSVD']['pois']=pois_accuracy

print("Poisoned test acc:", pois_accuracy)

In [ ]:
pois_vals=pois_model.gcn.feature_vals

In [ ]:
for k,v in pois_vals.items():
    print(v.shape)
    pois_vals[k]=v.cpu()

In [ ]:
import numpy as np
np.savez('feature_vals/gcnsvd_gp_'+ptb_value+'.npz', **pois_vals)

### Evasion global

In [ ]:
def loss_fn(A_flip):
    A_diff = A_flip * (1 - 2 * A)

    A_pert = A_low_rank + A_diff

    ############# Aux-Attack #############
    model = aux_model.sub(exclude=["low_rank"])

    scores = model(A_pert, X)
    return gb.metric.margin(scores[test_nodes, :], y[test_nodes]).tanh().mean()

def grad_fn(A_flip):
    return torch.autograd.grad(loss_fn(A_flip), A_flip)[0]

In [ ]:
########### PGD for Aux-Attack ###########
pert, _ = gb.attack.proj_grad_descent(A.shape, True, A.device, budget, grad_fn,\
                                      loss_fn, base_lr=0.1)

In [ ]:
A_pert = A + gb.pert.edge_diff_matrix(pert, A)
print("Adversarial edges:", pert.shape[0])
evas_accuracy = gb.metric.accuracy(aux_model(A_pert, X)[test_nodes], y[test_nodes]).item()
accuracy_dict['GCNSVD']['evas']=evas_accuracy

print("Evasion test acc: ", evas_accuracy)

In [ ]:
aux_model(A_pert,X)

In [ ]:
aux_model(A_pert,X)
evasion_vals=aux_model.gcn.feature_vals

In [ ]:
for k,v in evasion_vals.items():
    print(v.shape)
    evasion_vals[k]=v.cpu()

In [ ]:
import numpy as np
np.savez('feature_vals/gcnsvd_ge_'+ptb_value+'.npz', **evasion_vals)

In [ ]:
del clean_vals
del evasion_vals
del pois_vals

## GNNGuard

In [ ]:
rank = 50
fit_kwargs = dict(lr=1e-2, weight_decay=5e-4)

def make_model(div_limit=1e-6):
    return gb.model.GNNGuard(n_feat=D, n_class=C, hidden_dims=[64], dropout=0.5, div_limit=div_limit).cuda()

aux_model = make_model()
aux_model.fit((A, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs)

In [ ]:
clean_accuracy = gb.metric.accuracy(aux_model(A, X)[test_nodes], y[test_nodes]).item()
accuracy_dict['GNNGuard']['clean']=clean_accuracy

print("Clean test acc:   ", clean_accuracy)

In [ ]:
clean_vals = aux_model.feature_vals

In [ ]:
for k,v in clean_vals.items():
    print(v.shape)
    clean_vals[k]=v.cpu()

In [ ]:
import numpy as np
np.savez(f'feature_vals/gnnguard_clean_{ptb_value}.npz', **clean_vals)

### Poisoning global

In [ ]:
def loss_fn(A_flip):
    A_diff = A_flip * (1 - 2 * A)
    A_pert = A + A_diff

    ########## w/ real div_limit #########
    alteration = dict()
    ######################################

    ############# Meta-Attack ############
    model = make_model(**alteration)
    model.fit((A_pert, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs, max_epochs=50, differentiable=A_pert.requires_grad)
    scores = model(A_pert, X)
    ######################################

    return gb.metric.margin(scores[test_nodes, :], y[test_nodes]).tanh().mean()

In [ ]:
def grad_fn(A_flip):
    return torch.autograd.grad(loss_fn(A_flip), A_flip)[0]

In [ ]:
pert, _ = gb.attack.proj_grad_descent(A.shape, True, A.device, budget, grad_fn, loss_fn,\
                                      base_lr=0.1, grad_clip=0.1)
A_pert = A + gb.pert.edge_diff_matrix(pert, A)

In [ ]:
pois_model = make_model()
pois_model.fit((A_pert, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs)
pois_accuracy = gb.metric.accuracy(pois_model(A_pert, X)[test_nodes], y[test_nodes]).item()
accuracy_dict['GNNGuard']['pois']=pois_accuracy

print("Poisoned test acc:", pois_accuracy)

In [ ]:
pois_vals=pois_model.feature_vals

In [ ]:
for k,v in pois_vals.items():
    print(v.shape)
    pois_vals[k]=v.cpu()

In [ ]:
import numpy as np
np.savez('feature_vals/gnnguard_gp_'+ptb_value+'.npz', **pois_vals)

### Evasion global

In [ ]:
def loss_fn(A_flip):
    A_diff = A_flip * (1 - 2 * A)
    A_pert = A + A_diff

    ########## w/ real div_limit #########
    alteration = dict()

    ############# Aux-Attack #############
    with gb.model.changed_fields(aux_model, **alteration):
        scores = aux_model(A_pert, X)

    return gb.metric.margin(scores[test_nodes, :], y[test_nodes]).tanh().mean()

def grad_fn(A_flip):
    return torch.autograd.grad(loss_fn(A_flip), A_flip)[0]

In [ ]:
pert, _ = gb.attack.proj_grad_descent(A.shape, True, A.device, budget, grad_fn, loss_fn, base_lr=0.1)

A_pert = A + gb.pert.edge_diff_matrix(pert, A)
print("Adversarial edges:", pert.shape[0])
evas_accuracy = gb.metric.accuracy(aux_model(A_pert, X)[test_nodes], y[test_nodes]).item()
accuracy_dict['GNNGuard']['evas']=evas_accuracy

print("Evasion test acc: ", evas_accuracy)

In [ ]:
aux_model(A_pert,X)
evasion_vals=aux_model.feature_vals

In [ ]:
for k,v in evasion_vals.items():
    print(v.shape)
    evasion_vals[k]=v.cpu()

In [ ]:
import numpy as np
np.savez('feature_vals/gnnguard_ge_'+ptb_value+'.npz', **evasion_vals)

In [ ]:
del clean_vals
del evasion_vals
del pois_vals

## ProGNN

In [ ]:
fit_kwargs = dict(gnn_lr=0.01,gnn_weight_decay=0.0005,adj_lr=0.01,adj_momentum=0.9,reg_adj_deviate=1.0)

def make_model(A):
    return gb.model.ProGNN(A, GCN(n_feat=D, n_class=C, bias=True, activation="relu", hidden_dims=[64],dropout=0.5)).cuda()


In [ ]:
aux_model = make_model(A)
model_args = filter_model_args(aux_model, A, X)
aux_model.fit(model_args, y, train_nodes, val_nodes, progress=True, **fit_kwargs)

In [ ]:
clean_vals = aux_model.feature_vals

In [ ]:
clean_accuracy = gb.metric.accuracy(aux_model(X)[test_nodes], y[test_nodes]).item()
accuracy_dict['ProGNN']['clean']=clean_accuracy

print("Clean test acc:   ", clean_accuracy)

In [ ]:
for k,v in clean_vals.items():
    print(v.shape)
    clean_vals[k]=v.cpu()

In [ ]:
import numpy as np
np.savez(f'feature_vals/prognn_clean_{ptb_value}.npz', **clean_vals)

### Poisoning global

In [ ]:
torch.cuda.empty_cache()

In [ ]:
fit_kwargs2 = dict(gnn_lr=0.01,gnn_weight_decay=0.0005,adj_lr=0.01,adj_momentum=0.9,reg_adj_deviate=1.0,\
                   adj_optim_interval = 2, reg_adj_l1 = 5e-4, reg_adj_nuclear = 0, reg_feat_smooth = 1e-3)

def loss_fn(A_flip):
    A_diff = A_flip * (1 - 2 * A)
    A_pert = A + A_diff

    ############# Meta-Attack ############
    model = make_model(A_pert)
    model_args = filter_model_args(model, A_pert, X)
    model.fit(model_args, y, train_nodes, val_nodes, progress=False, **fit_kwargs2, differentiable=A_pert.requires_grad)
    #model.fit(X, y, train_nodes, val_nodes, progress=True, **fit_kwargs)
    scores = model(X)
    ######################################

    return gb.metric.margin(scores[test_nodes, :], y[test_nodes]).tanh().mean()

In [ ]:
def grad_fn(A_flip):
    return torch.autograd.grad(loss_fn(A_flip), A_flip)[0]

In [ ]:
pert, _ = gb.attack.proj_grad_descent(A.shape, True, A.device, budget, grad_fn, loss_fn,\
                                      base_lr=0.1, grad_clip=0.1)
A_pert = A + gb.pert.edge_diff_matrix(pert, A)

In [ ]:
pois_model = make_model(A_pert)
model_args = filter_model_args(pois_model, A_pert, X)
pois_model.fit(model_args, y, train_nodes, val_nodes, progress=True, **fit_kwargs)
pois_accuracy = gb.metric.accuracy(pois_model(X)[test_nodes], y[test_nodes]).item()
accuracy_dict['ProGNN']['pois']=pois_accuracy

print("Poisoned test acc:", pois_accuracy)

In [ ]:
pois_vals=pois_model.feature_vals

In [ ]:
for k,v in pois_vals.items():
    print(v.shape)
    pois_vals[k]=v.cpu()

In [ ]:
import numpy as np
np.savez('feature_vals/prognn_gp_'+ptb_value+'.npz', **pois_vals)

### Evasion global

In [ ]:
def loss_fn(A_flip):
    A_diff = A_flip * (1 - 2 * A)
    A_pert = A + A_diff


    ############# Aux-Attack #############
    
    model = aux_model
    model.S = A_pert
    scores = model(X)
    return gb.metric.margin(scores[test_nodes, :], y[test_nodes]).tanh().mean()

def grad_fn(A_flip):
    return torch.autograd.grad(loss_fn(A_flip), A_flip)[0]

In [ ]:
pert, _ = gb.attack.proj_grad_descent(A.shape, True, A.device, budget, grad_fn, loss_fn, base_lr=0.1)

A_pert = A + gb.pert.edge_diff_matrix(pert, A)
print("Adversarial edges:", pert.shape[0])
evas_accuracy = gb.metric.accuracy(aux_model(X)[test_nodes], y[test_nodes]).item()
accuracy_dict['ProGNN']['evas']=evas_accuracy

print("Evasion test acc: ", evas_accuracy)

In [ ]:
aux_model(X)
evasion_vals=aux_model.feature_vals

In [ ]:
for k,v in evasion_vals.items():
    evasion_vals[k]=v.cpu()

In [ ]:
import numpy as np
np.savez('feature_vals/prognn_ge_'+ptb_value+'.npz', **evasion_vals)

In [ ]:
del clean_vals
del evasion_vals
del pois_vals

## GRAND

In [ ]:
model_kwargs1 = dict(hidden_dims=[64],dropout=0.5)
model_kwargs2 = dict(dropnode=0.5,mlp_input_dropout=0.5,order=2)

def make_model():
    return GRAND(MLP(n_feat=D, n_class=C, bias=True, **model_kwargs1),**model_kwargs2).cuda()

#aux_model = make_model()

In [ ]:
aux_model = make_model()

In [ ]:
fit_kwargs = dict(lr=0.1, weight_decay=1e-4)
aux_model.fit((A,X), y, train_nodes, val_nodes, progress=True, **fit_kwargs)

In [ ]:
clean_accuracy = gb.metric.accuracy(aux_model(A,X)[test_nodes], y[test_nodes]).item()
accuracy_dict['GRAND']['clean']=clean_accuracy

print("Clean test acc:   ", clean_accuracy)

In [ ]:
clean_vals = aux_model.mlp.feature_vals

In [ ]:
for k,v in clean_vals.items():
    clean_vals[k] = v.cpu()

In [ ]:
import numpy as np
np.savez(f'feature_vals/grand_clean_{ptb_value}.npz', **clean_vals)

### Poisoning global

In [ ]:
def loss_fn(A_flip):
    A_diff = A_flip * (1 - 2 * A)
    A_pert = A + A_diff

    ############# Meta-Attack ############
    model = make_model()
    model.fit((A_pert, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs,max_epochs=100, differentiable=A_pert.requires_grad)
    scores = model(A_pert, X)
    ######################################

    return gb.metric.margin(scores[test_nodes, :], y[test_nodes]).tanh().mean()

In [ ]:
def grad_fn(A_flip):
    return torch.autograd.grad(loss_fn(A_flip), A_flip)[0]

In [ ]:
pert, _ = gb.attack.proj_grad_descent(A.shape, True, A.device, budget, grad_fn, loss_fn,\
                                      base_lr=0.1, grad_clip=0.1)
A_pert = A + gb.pert.edge_diff_matrix(pert, A)

In [ ]:
pois_model = make_model()
pois_model.fit((A_pert, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs)
pois_accuracy = gb.metric.accuracy(pois_model(A_pert, X)[test_nodes], y[test_nodes]).item()
accuracy_dict['GRAND']['pois']=pois_accuracy

print("Poisoned test acc:", pois_accuracy)

In [ ]:
pois_vals = pois_model.mlp.feature_vals
pois_vals

In [ ]:
for k,v in pois_vals.items():
    pois_vals[k]=v.cpu()

In [ ]:
import numpy as np
np.savez('feature_vals/grand_gp_'+ptb_value+'.npz', **pois_vals)

### Evasion global

In [ ]:
def loss_fn(A_flip):
    A_diff = A_flip * (1 - 2 * A)
    A_pert = A + A_diff


    ############# Aux-Attack #############
    model = aux_model
    scores = model(A_pert, X)

    return gb.metric.margin(scores[test_nodes, :], y[test_nodes]).tanh().mean()

def grad_fn(A_flip):
    return torch.autograd.grad(loss_fn(A_flip), A_flip)[0]

In [ ]:
pert, _ = gb.attack.proj_grad_descent(A.shape, True, A.device, budget, grad_fn, loss_fn, base_lr=0.1)

A_pert = A + gb.pert.edge_diff_matrix(pert, A)
print("Adversarial edges:", pert.shape[0])
evas_accuracy = gb.metric.accuracy(aux_model(A_pert, X)[test_nodes], y[test_nodes]).item()
accuracy_dict['GRAND']['evas']=evas_accuracy

print("Evasion test acc: ", evas_accuracy)

In [ ]:
aux_model(A_pert,X)
evasion_vals = aux_model.mlp.feature_vals

In [ ]:
for k,v in evasion_vals.items():
    evasion_vals[k]=v.cpu()

In [ ]:
import numpy as np
np.savez('feature_vals/grand_ge_'+ptb_value+'.npz', **evasion_vals)

In [ ]:
del clean_vals
del evasion_vals
del pois_vals

## Save the accuracy

In [ ]:
accuracy_dict

In [ ]:
import numpy as np
!mkdir accuracy_vals
import pickle
save_path = 'accuracy_vals/'+ptb_value+'.pkl'
with open(save_path, 'wb') as file:
    pickle.dump(accuracy_dict, file)